In [8]:
import numpy as np
import json

datafile = '../data/housing.csv'
data = np.fromfile(datafile, sep=' ')
data

array([6.320e-03, 1.800e+01, 2.310e+00, ..., 3.969e+02, 7.880e+00,
       1.190e+01])

In [9]:
feature_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE',
                 'DIS', 'RAD', 'TAX', 'PATRATIO', 'B', 'LSTAT', 'MEDV']
feature_num = len(feature_names)
data = data.reshape([data.shape[0] // feature_num, feature_num])
data

array([[6.3200e-03, 1.8000e+01, 2.3100e+00, ..., 3.9690e+02, 4.9800e+00,
        2.4000e+01],
       [2.7310e-02, 0.0000e+00, 7.0700e+00, ..., 3.9690e+02, 9.1400e+00,
        2.1600e+01],
       [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 3.9283e+02, 4.0300e+00,
        3.4700e+01],
       ...,
       [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 3.9690e+02, 5.6400e+00,
        2.3900e+01],
       [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 3.9345e+02, 6.4800e+00,
        2.2000e+01],
       [4.7410e-02, 0.0000e+00, 1.1930e+01, ..., 3.9690e+02, 7.8800e+00,
        1.1900e+01]])

In [10]:
x = data[0]
print(x.shape)
print(x)

(14,)
[6.320e-03 1.800e+01 2.310e+00 0.000e+00 5.380e-01 6.575e+00 6.520e+01
 4.090e+00 1.000e+00 2.960e+02 1.530e+01 3.969e+02 4.980e+00 2.400e+01]


In [11]:
ratio = 0.8
offset = int(data.shape[0] * ratio)
trainning_data = data[:offset]
trainning_data.shape

(404, 14)

In [13]:
maximums, minimums = trainning_data.max(axis=0), trainning_data.min(axis=0)
for i in range(feature_num):
    data[:, i] = (data[:, i] - minimums[i]) / (maximums[i] - minimums[i])

In [16]:
def load_data(datafile):
    # 从文件导入数据
    # datafile = './work/housing.data'
    data = np.fromfile(datafile, sep=' ')

    # 每条数据包括14项，其中前面13项是影响因素，第14项是相应的房屋价格中位数
    feature_names = [ 'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', \
                      'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV' ]
    feature_num = len(feature_names)

    # 将原始数据进行Reshape，变成[N, 14]这样的形状
    data = data.reshape([data.shape[0] // feature_num, feature_num])

    # 将原数据集拆分成训练集和测试集
    # 这里使用80%的数据做训练，20%的数据做测试
    # 测试集和训练集必须是没有交集的
    ratio = 0.8
    offset = int(data.shape[0] * ratio)
    training_data = data[:offset]

    # 计算训练集的最大值，最小值
    maximums, minimums = training_data.max(axis=0), training_data.min(axis=0)

    # 对数据进行归一化处理
    for i in range(feature_num):
        data[:, i] = (data[:, i] - minimums[i]) / (maximums[i] - minimums[i])

    # 训练集和测试集的划分比例
    training_data = data[:offset]
    test_data = data[offset:]
    return training_data, test_data

In [17]:
training_data, test_data = load_data(datafile)
x = training_data[:, :-1]
y = training_data[:, -1:]

In [18]:
print(x[0])
print(y[0])

[0.         0.18       0.07344184 0.         0.31481481 0.57750527
 0.64160659 0.26920314 0.         0.22755741 0.28723404 1.
 0.08967991]
[0.42222222]


In [19]:
w = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, -0.1, -0.2, -0.3, -0.4, 0.0]
w = np.array(w).reshape([13, 1])

In [20]:
x1 = x[0]
t = np.dot(x1, w)
print(t)

[0.69474855]


In [21]:
b = -0.2
z = t + b
print(z)

[0.49474855]


In [25]:
class Network(object):
    def __init__(self, num_of_weights):
        np.random.seed(0)
        self.w = np.random.randn(num_of_weights, 1)
        self.b = 0.
    
    def forward(self, x):
        z = np.dot(x, self.w) + self.b
        return z
    
    def loss(self, z, y):
        error = z - y
        cost = error * error
        cost = np.mean(cost)
        return cost

In [26]:
net = Network(13)
x1 = x[0:3]
y1 = y[0:3]
z = net.forward(x1)
print('predict: ', z)
loss = net.loss(z, y1)
print('loss:', loss)

predict:  [[2.39362982]
 [2.46752393]
 [2.02483479]]
loss: 3.384496992612791


In [24]:
Loss = (y1 -z) * (y1 -z)
print(Loss)

[3.88644793]


In [1]:
import paddle
from paddle.nn import Linear
import paddle.nn.functional as F
import numpy as np
import os
import random

In [4]:
def load_data(f):
    data = np.fromfile(f, sep=' ', dtype=np.float32)
    
    # 每条数据包括14项，其中前面13项是影响因素，第14项是相应的房屋价格中位数
    feature_names = [ 'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV' ]
    feature_num = len(feature_names)
    
    data = data.reshape([data.shape[0] // feature_num, feature_num])
    
    # 将原数据集拆分成训练集和测试集
    # 这里使用80%的数据做训练，20%的数据做测试
    # 测试集和训练集必须是没有交集的
    ratio = 0.8
    offset = int(data.shape[0] * ratio)
    training_data = data[:offset]
    
    # 计算train数据集的最大值，最小值
    maximums, minimums = training_data.max(axis=0), training_data.min(axis=0)
    
    # 记录数据的归一化参数，在预测时对数据做归一化
    global max_values
    global min_values
    
    max_values = maximums
    min_values = minimums
    
    # 对数据进行归一化处理
    for i in range(feature_num):
        data[:, i] = (data[:, i] - min_values[i]) / (maximums[i] - minimums[i])
    
    training_data = data[:offset]
    test_data = data[offset:]
    
    return training_data, test_data

In [8]:
# 验证数据集读取程序的正确性
data_file = '../data/housing.csv'
training_data, test_data = load_data(data_file)
print(training_data.shape)
print(training_data[1, :])

(404, 14)
[2.35922547e-04 0.00000000e+00 2.62405723e-01 0.00000000e+00
 1.72839552e-01 5.47997713e-01 7.82698274e-01 3.48961979e-01
 4.34782617e-02 1.14822544e-01 5.53191364e-01 1.00000000e+00
 2.04470202e-01 3.68888885e-01]


In [9]:
class Regressor(paddle.nn.Layer):
    def __init__(self):
        super(Regressor, self).__init__()
        
        self.fc = Linear(in_features=13, out_features=1)
    
    def forward(self, inputs):
        x = self.fc(inputs)
        return x

In [10]:
model = Regressor()
model.train()
training_data, test_data = load_data(data_file)
opt = paddle.optimizer.SGD(learning_rate=0.01, parameters=model.parameters())

In [11]:
EPOCH_NUM = 10
BATCH_SIZE = 10

for epoch_id in range(EPOCH_NUM):
    np.random.shuffle(training_data)
    mini_batches = [training_data[k:k+BATCH_SIZE] for k in range(0, len(training_data), BATCH_SIZE)]
    for iter_id, mini_batch in enumerate(mini_batches):
        x = np.array(mini_batch[:, :-1])
        y = np.array(mini_batch[:, -1:])
        house_features = paddle.to_tensor(x)
        prices = paddle.to_tensor(y)
        
        predicts = model(house_features)
        
        loss = F.square_error_cost(predicts, label=prices)
        avg_loss = paddle.mean(loss)
        if iter_id % 20 == 0:
            print("epoch: {}, iter: {}, loss is: {}".format(epoch_id, iter_id, avg_loss.numpy()))
        
        avg_loss.backward()
        opt.step()
        opt.clear_grad()

epoch: 0, iter: 0, loss is: [0.18709671]
epoch: 0, iter: 20, loss is: [0.18732806]
epoch: 0, iter: 40, loss is: [0.03730242]
epoch: 1, iter: 0, loss is: [0.07558794]
epoch: 1, iter: 20, loss is: [0.26684335]
epoch: 1, iter: 40, loss is: [0.09141275]
epoch: 2, iter: 0, loss is: [0.02915162]
epoch: 2, iter: 20, loss is: [0.07933918]
epoch: 2, iter: 40, loss is: [0.03347169]
epoch: 3, iter: 0, loss is: [0.04780255]
epoch: 3, iter: 20, loss is: [0.13286522]
epoch: 3, iter: 40, loss is: [0.1979607]
epoch: 4, iter: 0, loss is: [0.10601435]
epoch: 4, iter: 20, loss is: [0.04706139]
epoch: 4, iter: 40, loss is: [0.08828487]
epoch: 5, iter: 0, loss is: [0.09954137]
epoch: 5, iter: 20, loss is: [0.08786356]
epoch: 5, iter: 40, loss is: [0.05110221]
epoch: 6, iter: 0, loss is: [0.07174417]
epoch: 6, iter: 20, loss is: [0.02342386]
epoch: 6, iter: 40, loss is: [0.04081912]
epoch: 7, iter: 0, loss is: [0.076805]
epoch: 7, iter: 20, loss is: [0.04391838]
epoch: 7, iter: 40, loss is: [0.05142963]
epo

In [12]:
paddle.save(model.state_dict(), 'LR_model.pdparams')
print("模型保存成功，模型参数保存在LR_model.pdparams中")

模型保存成功，模型参数保存在LR_model.pdparams中


In [13]:
def load_one_example():
    idx = np.random.randint(0, test_data.shape[0])
    idx = -10
    one_data, label = test_data[idx, :-1], test_data[idx, -1]
    one_data = one_data.reshape([1, -1])
    
    return one_data, label

In [14]:
model_dict = paddle.load('LR_model.pdparams')
model.load_dict(model_dict)
model.eval()

one_data, label = load_one_example()
one_data = paddle.to_tensor(one_data)
predict = model(one_data)

predict = predict * (max_values[-1] - min_values[-1]) + min_values[-1]
label = label * (max_values[-1] - min_values[-1]) + min_values[-1]

print("Inference result is {}, the corresponding label is {}".format(predict.numpy(), label))

Inference result is [[21.266832]], the corresponding label is 19.700000762939453
